In [10]:
type_num=15
cuda_num = str(3)
date='12.06'
trainingSet_path0= "/data/fuwen/SuWen/Bert-classification/data/task3/train_"+date+".txt"
valSet_path0= "/data/fuwen/SuWen/Bert-classification/data/task3/test_"+date+".txt"
zeng_path0= "../data/sentiment/quan_9/zeng.txt"
model_save_path='../result3/classifier_'+date
type_=['ICT', '新能源汽车', '生物医药', '医疗器械', '钢铁', 
       '能源', '工业机器人', '先进轨道交通', '其他', '数控机床', 
       '工业软件', '高端装备', '半导体', '人工智能', '稀土']
www=[2.980574324324324, 1.0, 4.089223638470452, 9.748618784530386, 7.02988047808765, 2.204247345409119, 2.892622950819672, 12.694244604316546, 11.724252491694353, 8.503614457831326, 11.80267558528428, 3.529, 4.433417085427136, 3.529, 3.529]


In [11]:
"""
手动实现transformer.models.bert.BertForSequenceClassification()函数
根据论文[How to Fine-Tune BERT for Text Classification（2019）](https://www.aclweb.org/anthology/P18-1031.pdf)
在分类问题上，把最后四层进行concat然后maxpooling 输出的结果会比直接输出最后一层的要好
这里进行实现测试

"""
import torch
import torch.nn as nn
from transformers import BertModel,BertTokenizer
import torch.nn.functional as F


class bert_lr_last4layer_Config(nn.Module):
    def __init__(self):
        self.bert_path = "../chinese-bert-wwm"
        self.config_path = "../chinese-bert-wwm/config.json"

        # self.tokenizer = BertTokenizer.from_pretrained(self.bert_path)
        self.hidden_size = 768
        self.num_labels = type_num
        # self.dropout_bertout = 0.2
        self.dropout_bertout = 0.5
        self.mytrainedmodel = "../result/bert_clf_model.bin"
        """
        current loss: 0.4363991916179657 	 current acc: 0.8125
        current loss: 0.1328232882924341 	 current acc: 0.9527363184079602
        current loss: 0.11797185830000853 	 current acc: 0.9585411471321695
        train loss:  0.11880445411248554 	 train acc: 0.9583704495516361
        valid loss:  0.1511497257672476 	 valid acc: 0.9431549028896258
        """

class bert_lr_last4layer(nn.Module):

    def __init__(self,config):
        super(bert_lr_last4layer, self).__init__()
        self.bert = BertModel.from_pretrained(config.bert_path,config = config.config_path)
        self.dropout_bertout = nn.Dropout(config.dropout_bertout)
        self.num_labels = config.num_labels
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.sigmoid=nn.Sigmoid() 
        for param in self.bert.parameters():
            param.requires_grad = True

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=True,
        return_dict=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # outputs = outputs[2] # [1]是pooled的结果 # [3]是hidden_states 12层
        hidden_states = outputs.hidden_states
        nopooled_output = torch.cat((hidden_states[9],hidden_states[10],hidden_states[11],hidden_states[12]),1)
        batch_size = nopooled_output.shape[0] # 32
        # print(batch_size)
        # print(nopooled_output.shape) # torch.Size([32, 400, 768])
        kernel_hight = nopooled_output.shape[1]
        pooled_output = F.max_pool2d(nopooled_output,kernel_size = (kernel_hight,1))
        # print(pooled_output.shape) # torch.Size([32, 1, 768])

        flatten = pooled_output.view(batch_size,-1)
        # print(flatten.shape) # [32,768]

        flattened_output = self.dropout_bertout(flatten)

        logits = self.classifier(flattened_output)
        logits=self.sigmoid(logits)
        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return loss,logits

In [12]:
class Config(object):
    def __init__(self):
        self.config_dict = {
            "data_path": {
                # "trainingSet_path": "../data/sentiment/sentiment.train0.data",
                # "valSet_path": "../data/sentiment/sentiment.valid0.data",
                "trainingSet_path": trainingSet_path0,
                "valSet_path": valSet_path0,
                "testingSet_path": "../data/sentiment/sentiment.test0.data",
                "zeng_path": zeng_path0
            },

            "BERT_path": {
                "file_path": '../chinese-bert-wwm/',
                "config_path": '../chinese-bert-wwm/',
                "vocab_path": '../chinese-bert-wwm/',
            },

            "training_rule": {
                "max_length": 100,  # 输入序列长度，别超过512
                "hidden_dropout_prob": 0.3,
                "num_labels": type_num,  # 几分类个数
                "learning_rate": 1e-5,
                "weight_decay": 1e-2,
                "batch_size": 16
            },

            "result": {
                "model_save_path": '../result/bert_clf_model.bin',
                "config_save_path": '../result/bert_clf_config.json',
                "vocab_save_path": '../result/bert_clf_vocab.txt'
            }
        }

    def get(self, section, name):
        return self.config_dict[section][name]

In [13]:
import torch
from torch.utils.data import Dataset
import pandas as pd

class SentimentDataset(Dataset):
    def __init__(self, path_to_file):
#         print(path_to_file)
        self.dataset = pd.read_csv(path_to_file, sep="\t", names=["text", "label"])
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        text = self.dataset.loc[idx, "text"]
        label = self.dataset.loc[idx, "label"]
        sample = {"text": text, "label": label}
        # print(sample)
        return sample

def convert_text_to_ids(tokenizer, text, max_len=100):
    if isinstance(text, str):
        tokenized_text = tokenizer.encode_plus(text, max_length=max_len, add_special_tokens=True, truncation=True)
        input_ids = tokenized_text["input_ids"]
        token_type_ids = tokenized_text["token_type_ids"]
    elif isinstance(text, list):
        input_ids = []
        token_type_ids = []
        for t in text:
            tokenized_text = tokenizer.encode_plus(t, max_length=max_len, add_special_tokens=True, truncation=True)
            input_ids.append(tokenized_text["input_ids"])
            token_type_ids.append(tokenized_text["token_type_ids"])
    else:
        print("Unexpected input")
    return input_ids, token_type_ids

def seq_padding(tokenizer, X):
    pad_id = tokenizer.convert_tokens_to_ids("[PAD]")
    if len(X) <= 1:
        return torch.tensor(X)
    L = [len(x) for x in X]
    ML = max(L)
    X = torch.Tensor([x + [pad_id] * (ML - len(x)) if len(x) < ML else x for x in X])
    return X

In [14]:
import random

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from sklearn import metrics
import numpy
import transformers
from transformers import AdamW

In [15]:
class transformers_bert_binary_classification(object):
    def __init__(self):
        self.config = Config()
        self.device_setup()

    def device_setup(self):
        """
        设备配置并加载BERT模型
        :return:
        """
        self.freezeSeed()
        # 使用GPU，通过model.to(device)的方式使用
        device_s = "cuda:" + cuda_num
        self.device = torch.device(device_s if torch.cuda.is_available() else "cpu")

        # import os
        # result_dir = "../result"
        # MODEL_PATH = self.config.get("BERT_path", "file_path")
        # config_PATH = self.config.get("BERT_path", "config_path")
        vocab_PATH = self.config.get("BERT_path", "vocab_path")

        # num_labels = self.config.get("training_rule", "num_labels")
        # hidden_dropout_prob = self.config.get("training_rule", "hidden_dropout_prob")

        # 通过词典导入分词器
        self.tokenizer = transformers.BertTokenizer.from_pretrained(vocab_PATH)
        # self.model_config = BertConfig.from_pretrained(config_PATH, num_labels=num_labels,
        #                                                hidden_dropout_prob=hidden_dropout_prob)
        # self.model = BertForSequenceClassification.from_pretrained(MODEL_PATH, config=self.model_config)
        """
        train loss:  0.10704718510208534 	 train acc: 0.9637151849872321
        valid loss:  0.17820182011222863 	 valid acc: 0.9459971577451445
        """
        # 如果想换模型，换成下边这句子
        # bert+lr 跟官方方法差不都
        # self.model = bert_lr(bert_lr_Config())
        # self.model = bert_cnn(bert_cnn_Config())
        self.model = bert_lr_last4layer(bert_lr_last4layer_Config())

        self.model.to(self.device)

    def model_setup(self, zeng=0):
        weight_decay = self.config.get("training_rule", "weight_decay")
        learning_rate = self.config.get("training_rule", "learning_rate")
        print("**model_setup:")
        print("zeng",zeng)
        if zeng == 1:
            learning_rate = learning_rate * 2
        # 定义优化器和损失函数
        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': weight_decay},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
             'weight_decay': 0.0}
        ]
        self.optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
        self.criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(np.array(www)).float())
        self.criterion.to(self.device)

    def get_data(self):
        """
        读取数据
        :return:
        """
        train_set_path = self.config.get("data_path", "trainingSet_path")
        valid_set_path = self.config.get("data_path", "valSet_path")
        batch_size = self.config.get("training_rule", "batch_size")
        zeng_set_path = self.config.get("data_path", "zeng_path")
        print(train_set_path,valid_set_path,batch_size,zeng_set_path)

        # 数据读入
        # 加载数据集
        sentiment_train_set = SentimentDataset(train_set_path)
        sentiment_train_loader = DataLoader(sentiment_train_set, batch_size=batch_size, shuffle=True, num_workers=2)
        sentiment_valid_set = SentimentDataset(valid_set_path)
        sentiment_valid_loader = DataLoader(sentiment_valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

        sentiment_zeng_set = SentimentDataset(zeng_set_path)
        sentiment_zeng_loader = DataLoader(sentiment_zeng_set, batch_size=batch_size, shuffle=True, num_workers=2)

        return sentiment_train_loader, sentiment_valid_loader, sentiment_zeng_loader

    def train_an_epoch(self, iterator, zeng=0):
        print("**train_an_epoch")
        print("zeng",zeng)
        self.model_setup(zeng)
        epoch_loss = 0
        epoch_acc = 0

        for i, batch in enumerate(iterator):
            label = batch["label"]
            text = batch["text"]
            # print(label)
            input_ids, token_type_ids = convert_text_to_ids(self.tokenizer, text)
            input_ids = seq_padding(self.tokenizer, input_ids)
            token_type_ids = seq_padding(self.tokenizer, token_type_ids)
            # 标签形状为 (batch_size, 1)
            label = label.unsqueeze(1)
            # 需要 LongTensor
            input_ids, token_type_ids, label = input_ids.long(), token_type_ids.long(), label.long()
            # 梯度清零
            self.optimizer.zero_grad()
            # 迁移到GPU
            input_ids, token_type_ids, label = input_ids.to(self.device), token_type_ids.to(self.device), label.to(
                self.device)
            output = self.model(input_ids=input_ids, token_type_ids=token_type_ids, labels=label)  # 这里不需要labels
            # BertForSequenceClassification的输出loss和logits
            # BertModel原本的模型输出是last_hidden_state，pooler_output
            # bert_cnn的输出是[batch_size, num_class]
            # print(numpy.array(torch.tensor(output).cpu()).shape)

            y_pred_prob = output[1]
            y_pred_label = y_pred_prob.argmax(dim=1)

            # 计算loss
            # 这个 loss 和 output[0] 是一样的
            loss = self.criterion(y_pred_prob.view(-1, type_num), label.view(-1))  # 多分类改这里
            # loss = output[0]
            # 计算acc
            acc = ((y_pred_label == label.view(-1)).sum()).item()
            # 反向传播
            loss.backward()
            self.optimizer.step()
            # epoch 中的 loss 和 acc 累加
            epoch_loss += loss.item()
            epoch_acc += acc
            if i % 200 == 0:
                print("current loss:", epoch_loss / (i + 1), "\t", "current acc:", epoch_acc / ((i + 1) * len(label)))
        return epoch_loss / len(iterator), epoch_acc / len(iterator.dataset.dataset)

    def evaluate(self, iterator,xian):
        self.model.eval()
        epoch_loss = 0
        epoch_acc = 0
        y_pred_label_all = []
        label_all = []
        with torch.no_grad():
            for _, batch in enumerate(iterator):
                label = batch["label"]
                text = batch["text"]

                input_ids, token_type_ids = convert_text_to_ids(self.tokenizer, text)
                input_ids = seq_padding(self.tokenizer, input_ids)
                token_type_ids = seq_padding(self.tokenizer, token_type_ids)
                label = label.unsqueeze(1)
                input_ids, token_type_ids, label = input_ids.long(), token_type_ids.long(), label.long()
                input_ids, token_type_ids, label = input_ids.to(self.device), token_type_ids.to(self.device), label.to(
                    self.device)
                output = self.model(input_ids=input_ids, token_type_ids=token_type_ids, labels=label)
                # 更改了以下部分
                # y_pred_label = output[1].argmax(dim=1)
                y_pred_prob = output[1]
                
                y_pred_label=[]
                for i in y_pred_prob:
                    if max(i)>xian:
                        y_pred_label.append(i.argmax(dim=0))
                    else:
                        y_pred_label.append(torch.tensor(8).to(self.device))
#                         print(y_pred_label)
#                 y_pred_label=np.array(y_pred_label)
                y_pred_label=torch.tensor(y_pred_label).to(self.device)
    
                loss = output[0]
                # loss = self.criterion(y_pred_prob.view(-1, 2), label.view(-1))
                acc = ((y_pred_label == label.view(-1)).sum()).item()
                y_pred_label_all += y_pred_label.tolist()
                label_all += label.view(-1).tolist()

                epoch_loss += loss.item()
                epoch_acc += acc
        target_names=['ICT', '新能源汽车', '生物医药', '医疗器械', '钢铁', 
                       '能源', '工业机器人', '先进轨道交通', '其他', '数控机床', 
                       '工业软件', '高端装备', '半导体', '人工智能', '稀土']
        print(metrics.classification_report(label_all, y_pred_label_all,target_names=target_names))
        print("准确率:", metrics.accuracy_score(label_all, y_pred_label_all))
        return epoch_loss / len(iterator), epoch_acc / len(iterator.dataset.dataset)
    
    def evval(self,xian):
        sentiment_train_loader, sentiment_valid_loader, sentiment_zeng_loader= self.get_data()
        valid_loss, valid_acc = self.evaluate(sentiment_valid_loader,xian)
        print("valid loss: ", valid_loss, "\t", "valid acc:", valid_acc)

    def train(self, epochs, zeng=0):
        sentiment_train_loader, sentiment_valid_loader, sentiment_zeng_loader = self.get_data()
        best_valid_acc=0
        for i in range(epochs):
            print('____________________________________________________________________________________')
            print('____________________________________________________________________________________')
            print('epochs:', i)
            print('____________________________________________________________________________________')
            print('____________________________________________________________________________________')
            print('____train____')
            if zeng == 0:
                train_loss, train_acc = self.train_an_epoch(sentiment_train_loader)
            else:
                train_loss, train_acc = self.train_an_epoch(sentiment_zeng_loader, 1)
            print("train loss: ", train_loss, "\t", "train acc:", train_acc)
            print('____evaluate____')
            valid_loss, valid_acc = self.evaluate(sentiment_valid_loader,0.8)
            if valid_acc>best_valid_acc:
                best_valid_acc=valid_acc
                torch.save(classifier, model_save_path+'_old_'+str(valid_loss)+'_'+str(valid_acc))
            print("valid loss: ", valid_loss, "\t", "valid acc:", valid_acc)
        # self.save_model()

    def save_model(self):
        model_save_path = self.config.get("result", "model_save_path")
        config_save_path = self.config.get("result", "config_save_path")
        vocab_save_path = self.config.get("result", "vocab_save_path")

        model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
        torch.save(model_to_save.state_dict(), model_save_path)
        # model_to_save.config.to_json_file(config_save_path) # !!!'bert_lr' object has no attribute 'config'
        # self.tokenizer.save_vocabulary(vocab_save_path)
        print("model saved...")

    def predict(self, sentence):
        # self.model.setup()
        self.model_setup()
        self.model.eval()
        # 转token后padding
        input_ids, token_type_ids = convert_text_to_ids(self.tokenizer, sentence)
        input_ids = seq_padding(self.tokenizer, [input_ids])
        token_type_ids = seq_padding(self.tokenizer, [token_type_ids])
        # 需要 LongTensor
        input_ids, token_type_ids = input_ids.long(), token_type_ids.long()
        # 梯度清零
        self.optimizer.zero_grad()
        # 迁移到GPU
        input_ids, token_type_ids = input_ids.to(self.device), token_type_ids.to(self.device)
        output = self.model(input_ids=input_ids, token_type_ids=token_type_ids)
        # y_pred_prob:各个类别的概率
        y_pred_prob = output[0]
        # 取概率最大的标签
        y_pred_label = y_pred_prob.argmax(dim=1)

        # 将torch.tensor转换回int形式
        return y_pred_prob, y_pred_label.item()

    def freezeSeed(self):
        seed = 1
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)  # Numpy module.
        random.seed(seed)  # Python random module.
        torch.manual_seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

In [16]:
classifier = transformers_bert_binary_classification()
classifier.train(10,zeng=0)

/data/fuwen/SuWen/Bert-classification/data/task3/train_12.06.txt /data/fuwen/SuWen/Bert-classification/data/task3/test_12.06.txt 16 ../data/sentiment/quan_9/zeng.txt
____________________________________________________________________________________
____________________________________________________________________________________
epochs: 0
____________________________________________________________________________________
____________________________________________________________________________________
____train____
**train_an_epoch
zeng 0
**model_setup:
zeng 0
current loss: 2.8721139430999756 	 current acc: 0.0
current loss: 2.651389962998196 	 current acc: 0.13805970149253732
current loss: 2.4719861082899897 	 current acc: 0.34008728179551123
current loss: 2.364405526099308 	 current acc: 0.4449875207986689
train loss:  2.3542552398648233 	 train acc: 0.452165256346441
____evaluate____
              precision    recall  f1-score   support

         ICT       0.54      0.49   

current loss: 1.8898637592296632 	 current acc: 0.8965266222961731
train loss:  1.8907422833381944 	 train acc: 0.8963663514186162
____evaluate____
              precision    recall  f1-score   support

         ICT       0.75      0.68      0.71       346
       新能源汽车       0.90      0.84      0.87      1027
        生物医药       0.98      0.96      0.97       264
        医疗器械       0.91      0.89      0.90       108
          钢铁       0.94      0.98      0.96       155
          能源       0.89      0.87      0.88       489
       工业机器人       0.76      0.85      0.81       371
      先进轨道交通       0.90      0.99      0.94       100
          其他       0.27      0.40      0.32        86
        数控机床       0.62      0.74      0.67       113
        工业软件       0.61      0.63      0.62        89
        高端装备       0.91      0.77      0.84       302
         半导体       0.72      0.80      0.76       245
        人工智能       0.57      0.65      0.61       308
          稀土       0.99      0.93      0.

current loss: 1.8175512552261353 	 current acc: 1.0
current loss: 1.8458529206650767 	 current acc: 0.9617537313432836
current loss: 1.8469971257849525 	 current acc: 0.962281795511222
current loss: 1.8478520212078253 	 current acc: 0.9600665557404326
train loss:  1.8478444715973679 	 train acc: 0.9600796416127426
____evaluate____
              precision    recall  f1-score   support

         ICT       0.59      0.78      0.67       346
       新能源汽车       0.88      0.87      0.88      1027
        生物医药       0.99      0.96      0.98       264
        医疗器械       0.88      0.94      0.91       108
          钢铁       0.94      0.98      0.96       155
          能源       0.88      0.88      0.88       489
       工业机器人       0.79      0.81      0.80       371
      先进轨道交通       0.92      0.98      0.95       100
          其他       0.23      0.28      0.25        86
        数控机床       0.53      0.76      0.63       113
        工业软件       0.76      0.62      0.68        89
        高端装备      

In [17]:
device_s = "cuda:" + cuda_num
classifier=torch.load(model_save_path+'_old_1.9624433676401773_0.8276422764227642',map_location=torch.device(device_s if torch.cuda.is_available() else "cpu"))


In [18]:
classifier.evval(0.97)

/data/fuwen/SuWen/Bert-classification/data/task3/train_12.06.txt /data/fuwen/SuWen/Bert-classification/data/task3/test_12.06.txt 16 ../data/sentiment/quan_9/zeng.txt
              precision    recall  f1-score   support

         ICT       0.72      0.76      0.74       346
       新能源汽车       0.91      0.87      0.89      1027
        生物医药       0.99      0.94      0.96       264
        医疗器械       0.89      0.90      0.89       108
          钢铁       0.96      0.98      0.97       155
          能源       0.90      0.88      0.89       489
       工业机器人       0.79      0.83      0.81       371
      先进轨道交通       0.94      0.97      0.96       100
          其他       0.18      0.45      0.26        86
        数控机床       0.76      0.63      0.69       113
        工业软件       0.68      0.62      0.65        89
        高端装备       0.89      0.77      0.83       302
         半导体       0.83      0.71      0.76       245
        人工智能       0.61      0.66      0.63       308
          稀土       1.00

In [18]:
import torch
import jieba

device = torch.device("cpu")

classifier= torch.load(model_save_path,map_location=device)
classifier.device=torch.device('cpu')
ty = ['ICT', '新能源汽车', '生物医药', '医疗器械', '钢铁', 
       '能源', '工业机器人', '先进轨道交通', '其他', '数控机床', 
      '工业软件', '高端装备', '半导体', '人工智能', '稀土']
# print(classifier1.predict("『巴西』圣保罗城际铁路听证会延期至10月15日"))  # 0
# print(classifier1.predict("永恒力叉车入驻京东工业品 载重2吨的叉车设备也能线上采购"))  # 0

def read_list(text_path):
    lsit = []
    with open('%s' % text_path, 'r', encoding="utf8") as f:  # 打开一个文件只读模式
        line = f.readlines()  # 读取文件中的每一行，放入line列表中
        for line_list in line:
            lsit.append(line_list.replace('\n', ''))
    return lsit


def test():
    test_list = read_list('test.txt')
    for i in test_list:
        re = classifier.predict(i)  # 0
        result1 = re[1]
        result2 = re[0].tolist()

        if result2[0][result1] < 3.5:
            print(i, '\n', result1, '***** 其他 ***** 原预测:',ty[result1], result2[0][result1], '\n', result2[0], '\n')
        else:
            print(i, '\n', result1, ty[result1], result2[0][result1], '\n', result2[0], '\n')

test()

**model_setup:
zeng 0
e络盟社区发布新一期3D打印电子书 
 13 ***** 其他 ***** 原预测: 人工智能 3.3167057037353516 
 [-1.062137484550476, 0.29658377170562744, -3.6369457244873047, -1.4583207368850708, -2.523650884628296, -0.7532398104667664, -0.3904249370098114, -2.315615653991699, 2.3418962955474854, -2.028196334838867, -0.7419131398200989, 0.6681588292121887, 1.783487319946289, 3.3167057037353516, -0.5266007781028748] 

**model_setup:
zeng 0
2021中国国际消费电子博览会和青岛国际软件融合创新博览会开幕 
 13 人工智能 4.813075542449951 
 [-0.519827663898468, -0.06771048903465271, -3.2770726680755615, -2.5551161766052246, -3.226983070373535, -0.6945168972015381, -1.2149577140808105, -0.789153516292572, 2.3825902938842773, -1.678403377532959, 0.4534844756126404, 2.0999059677124023, -1.2294193506240845, 4.813075542449951, -1.8824306726455688] 

**model_setup:
zeng 0
4D打印软体机器人：打印出来即可工作 
 6 ***** 其他 ***** 原预测: 工业机器人 3.4218926429748535 
 [0.9916276931762695, -0.24243353307247162, -3.3082709312438965, 0.7730696201324463, -3.057868003845215, -1.5593171

中国商飞：6架C919国产大飞机正试验试飞，CR929研制进展顺利 
 10 ***** 其他 ***** 原预测: 工业软件 2.15157413482666 
 [0.2601698040962219, 1.1562528610229492, -2.5020039081573486, -0.719383180141449, -2.545905113220215, -0.43877872824668884, 1.4587992429733276, 0.136790469288826, -0.12382601201534271, -2.3512167930603027, 2.15157413482666, -1.4753119945526123, -1.1621516942977905, 0.931695818901062, -2.004362106323242] 

**model_setup:
zeng 0
库克被曝曾斥资1010万美元在加州购买豪宅面积 
 0 ICT 5.229444980621338 
 [5.229444980621338, 0.5307253003120422, -2.9565341472625732, -1.1298730373382568, -2.4798758029937744, -1.0669795274734497, -0.3427412807941437, -3.368483304977417, 1.946248173713684, 0.2853141725063324, -2.1717116832733154, -2.42659330368042, 1.378482460975647, -0.848236620426178, -1.4842661619186401] 

**model_setup:
zeng 0
京东联手东营打造“四新”新引擎，推动经济高质量发展 
 0 ***** 其他 ***** 原预测: ICT 2.6612131595611572 
 [2.6612131595611572, 0.10787053406238556, -3.5602681636810303, -1.2766337394714355, -3.744736909866333, -0.1547442525625229, 0.877112

医疗环境中设备的清洁与洗手同样重要 
 3 医疗器械 3.9782040119171143 
 [0.7428318858146667, -0.29262885451316833, -0.5204138159751892, 3.9782040119171143, -2.7050766944885254, -1.3139257431030273, 0.32242119312286377, -2.6076087951660156, 0.8146655559539795, -1.5607030391693115, -1.461133599281311, -1.2580697536468506, -1.0720912218093872, 0.04025343805551529, -0.3490920662879944] 

**model_setup:
zeng 0
盐湖股份拟1.2亿投建盐湖资源开发中试基地 
 8 ***** 其他 ***** 原预测: 其他 1.919896125793457 
 [0.9720925688743591, -0.1485719531774521, -2.056445598602295, -1.342449426651001, -2.515562057495117, 0.20252643525600433, 0.23671811819076538, -2.4533493518829346, 1.919896125793457, 1.0679724216461182, -0.9650934338569641, -1.3812153339385986, -0.7279885411262512, -0.8263260126113892, 1.3537533283233643] 

**model_setup:
zeng 0
沈阳新松机器人自动化股份有限公司荣获维科杯·OFweek2021中国工业自动化及数字化行业品牌影响力企业奖 
 6 工业机器人 7.2839226722717285 
 [0.45544731616973877, -0.8400704264640808, -3.524526357650757, -1.1515960693359375, -3.9764676094055176, -1.6001145839691162, 7.2

传出收购华晨“中华”品牌后！宝马将在沈阳追加投资250亿元 
 1 新能源汽车 7.333340167999268 
 [-0.3464181423187256, 7.333340167999268, -2.6362392902374268, -1.4853304624557495, -2.793823719024658, 0.39223670959472656, -1.3120195865631104, -2.282219886779785, -0.3798081874847412, -1.82382071018219, -0.649316668510437, -2.425593614578247, -0.778455913066864, 1.2658073902130127, -0.08457689732313156] 

**model_setup:
zeng 0
意法半导体推出具有内置智能的汽车高端驱动器 可节省高达40%的PCB面积 
 1 新能源汽车 4.681607723236084 
 [1.6019409894943237, 4.681607723236084, -2.9146368503570557, -2.2434678077697754, -3.25590443611145, -1.0740548372268677, -0.387160062789917, -2.7836618423461914, 0.37235429883003235, -0.32968002557754517, -2.107372283935547, -1.4913066625595093, 0.21589571237564087, 2.1955368518829346, -1.5029497146606445] 

**model_setup:
zeng 0
华为徐直军：部分汽车芯片获得了美国许可 
 0 ICT 6.452785015106201 
 [6.452785015106201, 2.558001756668091, -2.4507577419281006, -1.1234713792800903, -3.5352611541748047, -1.122612476348877, -1.6646451950073242, -2.776184797286987